<a href="https://colab.research.google.com/github/desultir/assign1/blob/master/assign2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
  
#!cd "/content/drive/My Drive/Colab Notebooks/PretrainedModels" && tar xvzf *.tar.gz
#!cd "/content/drive/My Drive/Colab Notebooks/Input" && tar xvzf test.tar.gz

Mounted at /content/drive


In [2]:
#install tensorboard tunnel
#https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-05-14 07:43:07--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.207.111.186, 34.206.9.96, 52.203.66.95, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.207.111.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16529980 (16M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.3’

ngrok-stable-linux- 100%[===================>]  15.76M  15.5MB/s    in 1.0s    

2019-05-14 07:43:08 (15.5 MB/s) - ‘ngrok-stable-linux-amd64.zip.3’ saved [16529980/16529980]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: asd
error:  invalid response [asd]
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: er
error:  invalid response [er]
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: xzc
error:  invalid response [xzc]
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: asd
error:  invalid response [asd]
replace ngrok? [y]es, [n]o, [A]

In [0]:
MODEL_DIR = "/content/drive/My Drive/Colab Notebooks/PretrainedModels/"
MODEL_NAME = "faster_rcnn_resnet50_coco_2018_01_28"

LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [4]:
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://506bbc08.ngrok.io


In [0]:
from matplotlib.image import imread
import PIL.Image
from PIL import ImageOps
import numpy as np
import io

def normalize_image(rgb):
  white_rgb = rgb - np.mean(rgb)
  white_rgb = white_rgb / np.max(np.abs(white_rgb))
  return white_rgb

def decompress_expand(jpg):
  #zero pad to 320x320
  encoded_jpg_io = io.BytesIO(jpg)
  image = PIL.Image.open(encoded_jpg_io)
  DESIRED_SIZE = 320
  widthpad = (DESIRED_SIZE - image.width)
  heightpad = (DESIRED_SIZE - image.height)
  padding = (widthpad//2, heightpad//2, widthpad - widthpad//2, heightpad-heightpad//2)
  padded = ImageOps.expand(image, padding)
  return padded


def load_and_preprocess(path):
  with tf.gfile.GFile(path, 'rb') as fp:
    jpg = fp.read()
  return decompress_expand(jpg)

def load_np(paths):
  return np.stack([imread(x) for x in paths])

In [0]:
import glob
import csv
import tensorflow as tf
import os
from tensorflow.core.framework import graph_pb2
from tensorflow.python.saved_model import loader
AUTOTUNE = tf.data.experimental.AUTOTUNE

#load train images
train_img_list = glob.glob(r"/content/drive/My Drive/Colab Notebooks/Input/train2014/*.jpg")
image_list = list(map(load_and_preprocess, train_img_list[:30]))
#train_img_list_ds = tf.data.Dataset.from_tensor_slices(image_list)
#load train labels
train_labels_filename = r"/content/drive/My Drive/Colab Notebooks/Input/train.txt"
with open(train_labels_filename) as f:
  reader = csv.reader(f, delimiter='\t')
  train_labels = {k:v for k, v in reader}

In [12]:
#load the trainable model from the checkpoint
sess = tf.Session()

saver = tf.train.import_meta_graph(os.path.join(MODEL_DIR,MODEL_NAME, 'model.ckpt.meta'))
class_weights = tf.get_variable("SecondStageBoxPredictor/ClassPredictor/weights", shape=(2048, 91), trainable=True)
class_bias = tf.get_variable("SecondStageBoxPredictor/ClassPredictor/biases", shape=(91), trainable=True)
sess.run(tf.global_variables_initializer())
saver.restore(sess, os.path.join(MODEL_DIR,MODEL_NAME, 'model.ckpt'))



ValueError: ignored

In [0]:
# write graph for tensorboard
summary_writer = tf.summary.FileWriter(LOG_DIR, sess.graph)

array([[-0.04742658,  0.03711859,  0.02226122, ..., -0.01608155,
        -0.01137163,  0.04715434],
       [ 0.00854041,  0.01166996,  0.04419908, ..., -0.04339767,
         0.02548472,  0.03276471],
       [ 0.01782445,  0.01630704, -0.00773155, ..., -0.03172974,
         0.00012942,  0.04512846],
       ...,
       [-0.03842142,  0.03758601,  0.01248769, ...,  0.01527385,
         0.03815636,  0.01854701],
       [-0.01291022, -0.0181416 , -0.00447982, ..., -0.05214845,
        -0.01356228, -0.03405831],
       [ 0.00751096,  0.02897915,  0.01423248, ...,  0.03020339,
        -0.03240707, -0.00874436]], dtype=float32)

In [0]:
stacked_imgs = np.stack(image_list)
#result = sess.run(("detection_classes:0", "detection_scores:0"), feed_dict={"image_tensor:0": stacked_imgs})

result = sess.run(("SecondStagePostprocessor/convert_scores:0", "detection_classes:0", "num_detections:0"), feed_dict={"image_tensor:0": stacked_imgs})

In [0]:
from scipy import stats
#np.argmax(result[0][3][:,3]) # prediction
#result[0][3] #prediction
preds = np.argmax(result[0], axis=2)
#result[1][3]

In [0]:
result[1][3] # classes

In [0]:
class relabeler(object):
  def __init__(self):
    self.labels = {}
     #mapping from cocolabel to tutorlabel + 1 (0 reserved for 'reject bounding box')
    self.labels[9] = 1 #boat 
    self.labels[10] = 5 #traffic light 
    self.labels[14] = 6 #parking meter
    self.labels[15] = 7 #bench
    self.labels[16] = 18 #bird
    self.labels[36] = 2 #snowboard
    self.labels[39] = 4 #baseball bat
    self.labels[41] = 3 #skateboard
    self.labels[44] = 10 #bottle
    self.labels[49] = 9 #knife
    self.labels[50] = 8 #spoon
    self.labels[52] = 15 #banana
    self.labels[54] = 13 #sandwich
    self.labels[57] = 14 #carrot
    self.labels[79] = 17 #oven
    self.labels[80] = 16 #toaster
    self.labels[85] = 12 #clock
    self.labels[90] = 11 #toothbrush   
    
  def fix(self, old):
    return self.labels.get(old, 0)
  
  def get_fix_fn(self):
    return np.vectorize(self.fix)
    

In [0]:
label_pl = tf.placeholder(tf.int32, [None, 1], name='labels')
#mapping from ben
fixfn = relabeler().get_fix_fn()
labels = fixfn(preds)
#TODO add mean here if batch training
loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=sess.graph.get_tensor_by_name("SecondStagePostprocessor/convert_scores:0"), labels=label_pl)

# Train op
with tf.name_scope("train"):
    # Get gradients of all trainable variables
    # TODO actually pass vars in, not their names
    var_list = [class_weights, class_biases]
    gradients = tf.gradients(loss, var_list)
    gradients = list(zip(gradients, var_list))
    learning_rate = 0.01

    # Create optimizer and apply gradient descent to the trainable variables
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.apply_gradients(grads_and_vars=gradients)

In [0]:
sess.graph.get_tensor_by_name("SecondStageBoxPredictor/ClassPredictor/weights:0")
#sess.graph.get_operation_by_name("SecondStageBoxPredictor/ClassPredictor/weights")